<a href="https://colab.research.google.com/github/j54854/myColab/blob/main/optWork_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最適化技術実験（第3部3回目）

---

## 学生番号

157xxxxx

---

## 氏名

青山　太郎

---





## 演習で利用するクラスの読込み

今回の演習課題でもJobsクラスと，前回の演習で作成したNodeクラスを利用しますので，最初にそれらを読み込んでおきましょう．まずはJobsクラスからです．




In [ ]:
import csv
import random
import sys
import time
import plotly.figure_factory as ff

class Jobs:
    def __init__(self, pt=None):
        if pt:  # pt must be 2-dimensional list of integers
            self.pt = pt  # pt[j][m]: processing time of job j on machine m
            self.J = len(self.pt)  # number of jobs
            self.M = len(self.pt[0])  # number of machines
        else:  # if pt is not given
            pass  # do nothing (pt, J, M sould be set later)

    def save_in_csv_file(self, filename):  # pt_jm is saved in a csv file
        with open(filename, 'w') as f:
            writer = csv.writer(f)
            for j in range(self.J):
                writer.writerow(self.pt[j])

    def set_by_csv_file(self, filename):  # pt, J & M are set by a csv file, where rows correespond to jobs and columns correspond to machines
        with open(filename) as f:
            reader = csv.reader(f)
            self.pt = [
                [int(pt) for pt in row] for row in reader
            ]
            self.J = len(self.pt)
            self.M = len(self.pt[0])

    def set_randomly(self, J, M):  # pt_jm is set randomly for specified J & M
        self.J = J
        self.M = M
        self.pt = [
            [random.randint(1, 60) for m in range(M)] for j in range(J)
        ]

    def make_schedule(self, seq=None):
        if not seq:  # if seq is not given, it is set randomly
            seq = list(range(self.J))  # seq = [0, 1, 2, ..., J]
            random.shuffle(seq)  # randomize the order of elements in seq
        self.st = [[0] *self.M for j in range(self.J)]  # start time of job j on machine m is initilized to be 0
        last_ct = [0 for m in range(self.M)]  # when each machine finishes the last job (& becomes available) is also set to be 0
        for j in seq:
            last_ct_j = 0  # when job j is completed on the last machine (& becomes ready) is initially set to be 0
            for m in range(self.M):
                self.st[j][m] = max(last_ct_j, last_ct[m])
                last_ct[m] = self.st[j][m] +self.pt[j][m]
                last_ct_j = self.st[j][m] +self.pt[j][m]
        print('Job sequence = ', seq)
        print('Makespan = {}'.format(last_ct_j))

    def draw_schedule(self):
        operations = []
        for j in range(self.J):
            for m in range(self.M):
                day = '2020-12-01 '  # dummy date
                st_h, st_m = divmod(self.st[j][m], 60)
                ct_h, ct_m = divmod(self.st[j][m] +self.pt[j][m], 60)
                st_daytime = day +'{:02}:{:02}'.format(st_h, st_m)
                ct_daytime = day +'{:02}:{:02}'.format(ct_h, ct_m)
                operations.append(
                    dict(Task='Machine'+str(m), Start=st_daytime, Finish=ct_daytime, Resource='Job'+str(j))
                )
        fig = ff.create_gantt(operations, index_col='Resource', show_colorbar=True, group_tasks=True)
        fig.show()

次にNodeクラスを読み込みます．なお，下記のサンプルコードは各メソッドの中身が記述されていないスケルトンですので，前々回の演習課題で作成したコードに置き換えてから実行するようにします．



In [ ]:
class Node:
    def __init__(self, jobs, seq=None):
        self.jobs = jobs  # all jobs (Jobs class instance)
        self.seq = []  # (partial) job sequence
        self.rest = list(range(jobs.J))  # unassigned jobs
        self.last_ct = [0] *jobs.M  # when each machine finishes the last job (& becomes available) is initially set to be 0
        if seq:
            self.set_seq(seq)

    @property
    def makespan(self):
        pass

    def assign(self, j):  # assign job j in rest
        pass

    def set_seq(self, seq):  # set (partial) job sequence at a time
        pass

    def is_complete(self):  # is this a full schedule?
        pass

## 厳密解法と近似解法

与えられたスケジューリング問題の最適解（今考えている例では，メイクスパンを最小にするジョブの順序）の1つを必ず出力する解法を厳密解法と呼びます．例えば，前回，前々回に実装した全数探索や分枝限定法は，厳密解法に分類されます．ここで扱っているフローショップスケジューリング問題を始めとして，多くのスケジューリング問題では，問題の規模に伴って必要な計算時間が爆発的に大きくなっていく現象（組合せ爆発）が生じるため，厳密解法は，比較的小規模な問題しか扱えないという限界を抱えています．

これに対して，必ず最適解を導くわけではありませんが，現実的な計算時間内になるべく良質な解を得ることを目指した解法を，近似解法と呼びます．近似解法の中には，得られる解と最適解の質の差の上限を理論的に保証できるものもありますが，一般には，その差の上限は必ずしも明らかではありません．この後者の，より一般的な近似解法を，特に，発見的手法（ヒューリスティクス）と呼ぶことがあります．多様なヒューリスティクスが提案されていますが，その最も基本的かつ汎用的な枠組みの1つとして，局所探索があります．今回はこの局所探索の実装に挑戦しましょう．




## 近傍，局所解，局所探索

ある（部分ではなく完全な）スケジュールが与えられたとき，何らかの意味でそれと似通ったスケジュールの集合をそのスケジュールの近傍と呼びます．そして，あるスケジュールの近傍に，そのスケジュールよりも良い（今の例では，メイクスパンの短い）スケジュールが存在しないとき，それを局所解，あるいは局所最適解と呼びます．

局所探索とは，可能なスケジュール全体の集合（解空間）を漏れなく探索するのではなく，あるスケジュール（初期解）から始め，その近傍だけを探索し，もしより良いスケジュールがあれば，暫定的に解をそれに更新する，そして，更新したスケジュールの近傍をまた探索する，という処理を繰り返すことによって，最終的に局所解に辿り着く方法です．

局所探索の実装に取り掛かる前に，まずは近傍を構成することを考えてみましょう．



## 順列の交換近傍と挿入近傍

ここで考えているフローショップスケジューリング問題では，個々の解（すなわちスケジュール）はそれぞれジョブの順序（順列）に対応しています．順列について様々な近傍が考えられますが，ここでは，それらの中でも最も基本的なものとして，交換近傍と挿入近傍の2つを実装してみます．

### 交換近傍

ある順列seqが与えられたとき，seqの中の任意の2つの要素の位置を入れ替えることによって得られる順列の集合を，seqの交換近傍と呼びます．例えば，seq=(0,4,1,3,2)だとしましょう．このとき，最初と最後の要素を入れ替えると，(2,4,1,3,0)という順列が得られます．したがって，(2,4,1,3,0)はseqの交換近傍に含まれる，ということになります．交換近傍に含まれる順列の数がJ(J-1)/2個になることを確認しておきましょう．

### 挿入近傍

一方，seqの中から1つの要素を取り出し，その取り出した要素を別の位置に挿入することによって得られる順列の集合を，seqの挿入近傍と呼びます．例えば上の例で，最初の要素を取り出してそれを最後尾に挿入すると，(4,1,3,2,0)という順列が得られます．したがって，(4,1,3,2,0)はseqの挿入近傍に含まれる，ということになります．挿入近傍に含まれる順列の数が(J-1)^2になることを確認しておきましょう．

## 演習課題1: 近傍の実装

### 課題1-1

順列seq（長さJのリスト）を渡すと，その交換近傍を順列のリスト（長さJのリストのリスト）として返す関数swap_neighbors()を完成させましょう．なお，関数の戻り値として返すリストneighborhoodに同じ順列が複数含まれることがないようにすること．


In [ ]:
def swap_neighbors(seq):
    neighborhood = []
    pass
    return neighborhood

### 課題1-2

上と同様にして，順列seqを渡すと，その挿入近傍を順列のリストとして返す関数insert_neighbors()を完成させよう．今回も，戻り値として返すリストneighborhoodに同じ順列が複数含まれることがないようにしよう．


In [ ]:
def insert_neighbors(seq):
    neighborhood = []
    pass
    return neighborhood

うまく実装できたかどうかを，例えば次のようにして確認しておこう．

In [ ]:
seq = [0,1,2,3]
print(swap_neighbors(seq))
print(insert_neighbors(seq))

## 局所探索のバリエーション

### 初期解の与え方

通常はランダムに生成します．また，異なる初期解で局所探索を繰り返し，得られた局所解のうち最も良いスケジュールを選択するという戦略をとることもよくあります．これは，多スタート局所探索と呼ばれています．

### 近傍の定義

上で見たように，最も基本的なものとして，交換近傍と挿入近傍があります．また，これらの他にも様々な近傍が提案されています．

### 移動戦略

次の2つが代表的な移動戦略です．

- 近傍内のすべての順列（スケジュール）を評価し，暫定解より良い解があればそれらのうち最良のものに移動する（なければ終了）．

- 暫定解よりも良い解が見つかり次第，すぐにそこに移動する（近傍内を評価し尽くしても良い解が見つからなければ終了）．なお，こちらの戦略をとる場合は，通常，近傍内の解はランダムな順序で評価します．


## 演習課題2: 局所探索の実装

### 課題2-1

近傍を調べ尽くしてからその中の最良解に移動するという移動戦略で局所探索を行う関数local_search_a()を完成させよう．交換近傍と挿入近傍のどちらを使ってもかまいません．なお，引数jobsはJobsクラスのインスタンス，seqは初期解を表す順列（長さJのリスト）で，戻り値は，局所解の順列best_seqとそれに対応するスケジュールのメイクスパンbest_msとします．関数の中で，Nodeクラスのインスタンスを利用するといいでしょう．

In [ ]:
def local_search_a(jobs, seq):
    best_seq = None  # best job sequence found so far
    best_ms = sys.maxsize  # its makespan
    pass
    return best_seq, best_ms

### 課題2-2

暫定解よりも良い解が見つかればすぐそれに移動するという移動戦略で局所探索を行う関数local_search_b()を完成させましょう．今回も，交換近傍と挿入近傍のどちらを使ってもかまいません．引数や戻り値は上と同じとする．近傍の中の解をチェックしていく順序がランダムになるようにすること．

In [ ]:
def local_search_b(jobs, seq):
    best_seq = None  # best job sequence found so far
    best_ms = sys.maxsize  # its makespan
    pass
    return best_seq, best_ms

これらの関数も，仕上がったら例えば次のようにして動作確認しておこう．

In [ ]:
pt = [
        [35, 7, 18, 20],
        [4, 60, 12, 10],
        [15, 15, 20, 30],
        [11, 25, 10, 35],
        [33, 1, 45, 12]
    ]
jobs = Jobs(pt)
seq = list(range(jobs.J))
random.shuffle(seq)
best_seq, best_ms = local_search_a(jobs, seq)
# best_seq, best_ms = local_search_b(jobs, seq)
jobs.make_schedule(best_seq)
jobs.draw_schedule()

## 考察課題3

最後に，今回の講義と課題に関連して，下記の項目について考察してみましょう．下の欄に直接書き込んで提出してください．

- しばしば局所探索の拡張として捉えられる近似解法のクラスとして，メタヒューリスティクスがあります．メタヒューリスティクスの具体的なアルゴリズムを1つ取り上げて，それが単純な局所探索と比較してどのような強みをもつかを考えてみよう．

### 課題3-1

この文を消して，ここに，自分の考察を記入してください．


## まとめ

演習課題のコードは，所定の箇所に（passを消してから）直接書き込んでください．提出前に必ず動作を確認しておくこと．動作しないコードは採点対象になりません．また，考察課題も，所定の欄に直接書き込んでもらえればOKです．

最後に，このGoogle Colabのノートブックを

★☆★ <b>ipynb形式</b> ★☆★

で保存したファイルをコースパワーから提出すること．提出の〆切は，

★☆★ <b>18:20</b> ★☆★

とします．